In [1]:
import uuid
from pathlib import Path

import dotenv
import wikipedia
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver

from open_deep_researcher.graph import builder

dotenv.load_dotenv()
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [3]:
TMP_DIR = Path("./tmp")


def download_wiki(query: str) -> Path:
    doc_dir = TMP_DIR / "docs"
    doc_dir.mkdir(exist_ok=True, parents=True)

    filename = doc_dir / f"{query.lower()}.txt"
    if filename.exists():
        return doc_dir

    with open(filename, "w", encoding="utf-8") as f:
        wikipedia.set_lang("ja")
        page = wikipedia.page(query)
        text = page.content
        f.write(text)
    return doc_dir


local_document_path = download_wiki("Python")


In [ ]:
# web search
thread = {
    "configurable": {
        "thread_id": str(uuid.uuid4()),
        "skip_human_feedback": True,
        "enable_deep_research": False,
        "search_source": "web",
        "web_search_config": {
            "provider": "tavily",
            "max_results": 5,
            "include_raw_content": False,
        },
    }
}

topic = "Python 1.0 がリリースされた年は？"
async for event in graph.astream({"topic": topic}, thread, stream_mode="updates"):
    print(event)
    print("\n")

In [ ]:
# local search
thread = {
    "configurable": {
        "thread_id": str(uuid.uuid4()),
        "skip_human_feedback": True,
        "enable_deep_research": False,
        "search_source": "local",
        "local_search_config": {
            "vector_store_path": (TMP_DIR / "vector_store").as_posix(),
            "local_document_path": local_document_path.as_posix(),
        },
    }
}

topic = "Python 1.0 がリリースされた年は？"
async for event in graph.astream({"topic": topic}, thread, stream_mode="updates"):
    print(event)
    print("\n")

In [ ]:
# hybrid search
thread = {
    "configurable": {
        "thread_id": str(uuid.uuid4()),
        "skip_human_feedback": True,
        "enable_deep_research": False,
        "search_source": "hybrid",
        "web_search_config": {
            "provider": "tavily",
            "max_results": 5,
            "include_raw_content": False,
        },
        "local_search_config": {
            "vector_store_path": (TMP_DIR / "vector_store").as_posix(),
            "local_document_path": local_document_path.as_posix(),
            "collection_name": "wiki",
        },
    }
}

topic = "Python 1.0 がリリースされた年は？"
async for event in graph.astream({"topic": topic}, thread, stream_mode="updates"):
    print(event)
    print("\n")

In [5]:
# clean up
import shutil

shutil.rmtree(TMP_DIR)
